# Export

> Exporting to python module

In [1]:
#| default_exp sync.export

In [2]:
#|export
import shlex
import os
import ast
import joblib
from pathlib import Path

from nbdev.processors import Processor, NBProcessor
from execnb.shell import CaptureShell
from execnb.nbio import new_nb, mk_cell, write_nb

## TODO

- read test and non-test pk files and print code to screen when valid=True
- in magic line, replace %% with #@@, and add #| export as first line

::: {.content-hidden}
## obtain_function_name_and_test_flag
::: 

In [3]:
#|export
def obtain_function_name_and_test_flag (line, cell):

    root = ast.parse (cell)
    name=[x.name for x in ast.walk(root) if isinstance (x, ast.FunctionDef)]
    argv = shlex.split(line, posix=(os.name == 'posix'))
    is_test = '--test' in argv
    
    if len(name)>0:
        function_name=name[0]
    else:        
        function_name=argv[0] if len(argv)>0 else ''
        if function_name.startswith('-'):
            function_name = ''

    if function_name=='':
        raise RuntimeError ("Couldn't find function name.")
    
    return function_name, is_test

::: {.content-hidden}
## FunctionNBProcessor
:::

In [4]:
#|export
class FunctionNBProcessor(Processor):
    def __init__ (
        self, 
        file_name_without_extension,
        code_cells_file_name=None,
        code_cells_path='.nbmodular',
        execute=True,
        nb_path=None,
        nbs_folder='nbs',
    ):
        self.code_cells_path=Path(code_cells_path)
        code_cells_file_name = file_name_without_extension if code_cells_file_name is None else code_cells_file_name
        path_to_code_cells_file = self.code_cells_path / f'{code_cells_file_name}.pk'
        test_path_to_code_cells_file = self.code_cells_path / f'test_{code_cells_file_name}.pk'
        if not path_to_code_cells_file.exists() and not test_path_to_code_cells_file.exists():
            nb_path = Path(nbs_folder) / f'{file_name_without_extension}.ipynb' if nb_path is None else nb_path
            if nb_path.exists() and execute:
                caputure_shell = CaptureShell ()
                caputure_shell.execute(nb_path, 'tmp.ipynb')
            elif not execute:
                raise RuntimeError ('Exported pickle files not found and execute is False')
            else:
                raise RuntimeError (f'Exported pickle files not found and notebook with path {nb_path} not found')
        
        self.code_cells = joblib.load (path_to_code_cells_file) if path_to_code_cells_file.exists() else {}
        self.test_code_cells = joblib.load (test_path_to_code_cells_file) if path_to_code_cells_file.exists() else {}

        self.function_names = {}
        self.test_function_names = {}
        self.cells = []
        self.dest_nb_path = Path(nbs_folder) / f'dest_{file_name_without_extension}.ipynb'
    
    def cell(self, cell):
        cell_lines = cell.source.splitlines()
        if len(cell_lines) > 0 and cell_lines[0].strip().startswith('%%'):
            line = cell_lines[0]
            cell = '\n'.join (cell_lines[1:])
            to_export = False
            if line.startswith('%%function') or line.startswith('%%method'):
                function_name, is_test = obtain_function_name_and_test_flag (line, cell)
                function_names = self.test_function_names if is_test else self.function_names
                if function_name in function_names:
                    function_names[function_name] += 1
                else:
                    function_names[function_name] = 0
                idx = function_names[function_name]
                print (f'{function_name}, {idx}, is test: {is_test}')
                code_cells = self.test_code_cells if is_test else self.code_cells
                code_cell = code_cells[function_name]
                print ('code:')
                print (code_cell.code, 'valid: ', code_cell.valid)
                if code_cell.valid:
                    cell = code_cell.code
                    to_export = True
            elif line.startswith ('%%include') or line.startswith ('%%class'):
                to_export = True
            line = line.replace ('%%', '#@@')
            cell = line + '\n' + cell
            if to_export:
                cell = '#|export\n' + cell
        self.cells.append (mk_cell (cell))
    def end(self): 
        nb = new_nb (self.cells)
        write_nb (nb, self.dest_nb_path)


::: {.content-hidden}
## process_notebook
:::

In [5]:
#|export
def process_notebook (debug=False):
    function_processor = FunctionNBProcessor (
        'test_export',
        nbs_folder='test_nbs',
    )
    NBProcessor ('./test_nbs/test_export.ipynb', function_processor).process()

In [6]:
process_notebook ()

add1, 0, is test: False


KeyError: 'add1'